# ch11. A foundational model and training loop
---
> - `DataLoader`로 데이터 로딩하기
> - CT 데이터 분류기 구현
> - 어플리케이션 기본 구조 설정
> - metric 로깅

#### 이 노트북은 코드 실행이 되지 않는다. 책에서 설명하는 부분들을 정리하기 위해 작성한 노트북이다.

이번 장에서는 분류 모델을 만드는 것과 훈련 루프를 만들어본다. 기존에 설명했던 전체 프로젝트에서는 아래 그림에 해당하는 부분이다.

![image](https://user-images.githubusercontent.com/76675506/190555112-8e0215b5-b1e0-4ce5-a6e5-dd4f7c94b234.png)

이번 장에서 구현할 사항을 자세하게 나타내면 아래 그림과 같다. 전체 구조는 다음과 같다.

- 데이터를 불러오고 모델을 초기화한다.
- epoch 별로 어느 정도 임의로 훈련한다.
    - `LunaDataset`에서 반환된 배치를 루프한다.
    - data-loader가 적합한 배치를 불러온다.
    - 배치를 분류 모델로 전달한다.
    - loss를 계산한다.
    - metric을 기록한다.
    - 가중치를 업데이트한다.
    - valid 배치를 루프한다.
    - 적합한 vliad 배치를 불러온다.
    - loss를 계산한다.
    - 결과를 기록한다.


![image](https://user-images.githubusercontent.com/76675506/190555189-eb7543ea-fee2-40f5-975f-cbfff9fd9a83.png)


지금부터 만들 training loop는 기존에 우리가 만들었던 training loop과 크게 두 가지 차이점이 있다.
1. 좀 더 정교하게 만든다.
   - 너무 단순하면 성능이 떨어지거나, 유지보수가 힘들어지거나, 뭘 하는지 설명하기 어려워진다.
2. train이 진행되는 동안 다양한 metric을 기록한다.
   - 프로젝트에 적합한 metric을 알 수 있다.

전체 프로젝트 입장에서 봤을 때도 구조적인 차이가 있는데, 바로 완전한 command-line 어플리케이션이라는 점이다. 즉 주피터나 쉘에서도 동작할 수 있게끔 설계한다.

In [2]:
import argparse
# code/p2_run_everything.ipynb
import datetime
import sys

import torch.cuda

from util.util import importstr
from util.logconf import logging
log = logging.getLogger('nb')


In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

NameError: name 'tf' is not defined

In [2]:
# code/p2_run_everything.ipynb
# 코랩으로 실행해야 함
def run(app, *argv):
    argv = list(argv)
    argv.insert(0, '--num-workers=4')
    log.info("Running: {} ({!r}).main()", format(app, argv))

    app_cls = importstr(*app.rsplit('.', 1))
    app_cls(argv).main()

    log.info("Finished: {}.{!r}).main()".format(app, argv))

In [3]:
# code/p2_run_everything.ipynb
run('p2ch11.training.LunaTrainingApp', '--epochs=1')

TypeError: format() argument 2 must be str, not list

아래 코드는 로깅을 위해 일반적으로 쓰이는 코드다. 다른 프로젝트를 위해 재사용할 수도 있다. 특히 `__init__`을 파싱하는 부분은 어플리케이션을 따로 configure할 수 있게 해준다.

In [ ]:
# training.py:31, class LunaTrainingApp
class LunaTraingApp:
    def __init__(self, sys_argv=None):
        if sys_argv is None:        # caller가 arg가 없다면 command-line에서 arg를 얻는다.
            sys_argv = sys.argv[1:]

        parser = argparse.ArgumentParser()
        parser.add_argument('--num-workers',
                            help='Number of worker processes for background data loading',
                            default=8,
                            type=int,
                            )
        #...line 63
        self.cli_args = parser.parse_args(sys_argv)
        self.time_str = datetime.datetime.now().strftime('%Y-%m-%d_%H. %M. %S') # timestamp
    def main(self):
        log.info("starting {}, {}". format(type(self).__name__, self.cli_args))

epcoh를 실행하기 전에 두가지 할일이 있다. 먼저 모델과 optimizer를 초기화해야 하는 것과 `Dataset`과 `DataLoader`를 초기화 해야 한다.
`LunaDataSet`은 epoch 마다 무작위 샘플을 정의하고 `DataLoader`는 데이터를 불러오고 프로젝트의 어플리케이션에 데이터를 제공한다.

![image](https://user-images.githubusercontent.com/76675506/190560426-182cfb99-8712-407c-9202-f1c1672e65d4.png)

이 챕터에서는 `LunaModel`을 블랙박스로 생각하고 시작해보자.

In [ ]:
# training.py:31, class LunaTrainingApp
class LunaTraingApp:
    def __init__(self, sys_argv=None):
        #...line 70
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")

        self.model = self.initModel()
        self.optimizer = self.initOptimizer()

    def initModel(self):
        model = LunaModel()
        if self.use_cuda:
            log.info("Using CUDA; {} devices.".foramt(torch.cuda.device_count()))
            if torch.cuda.device_count() > 1:
                model = nn.DataParall(model)
            model = model.to(self.device)
        return model


위 코드 16 번째 줄에 사용한 `DataParall`가 GPU를 병렬로 사용하기 제일 좋은 선택지는 아니다.

> `DataParall` vs `DistributedDataParall`
>  - `DataParall`: 간단하게 모델을 래핑해서 multiple GPU를 활용한다. 하지만 제한된 자원만 사용한다.
>  - `DistributedDataParall`: 하나 이상의 GPU나 machine을 사용하고 싶을 때 추천하는 래핑 클래스다. 하지만 설정이 복잡하기 때문에 여기서는 사용하지 않는다. 도큐먼트를 참조하자 https://pytorch.org/tutorials/intermediate/ddp_tutorial.html




optimizer를 만들기 전에 모델을 GPU로 옮겨야 한다. 그렇지 않으면 옵티마이저는 CPU에서 파라미터 값을 찾으려 한다.

이 프로젝트에서는 optimizer로 SGD를 사용한다. SGD와 모멘텀을 사용한다. learning rate을 SGD는 0.001, 모멘텀은 0.9로 하면 안전한 선택이다. 다양한 learning rate 값을 사용해보고 네트워크 사이즈 등을 조정하는걸 `hyperparameter search`라고 한다.

데이터를 불러올 때는 모델이 요구하는 데이터 형식으로 맞춰줘야 한다. 예를 들어 `torch.nn.Conv3d`는 (N, C, D, H, W): Number of sample, Channels per sample, Depth, Height, Wdith의 형태로 데이터를 넣어야 한다. 이를 맞추기 위해 `LunaDataset.__getitem__`에서 `ct_t.unsqueeze(0)`을 통해 차원을 맞춰줬다.

또 훈련을 진행할 때는 여러 개의 샘플(배치)을 동시에 처리해 병렬 연산을 진행한다. 이 부분은 파이토치의 `DataLoader`를 통해서 구현할 수 있다.

In [ ]:
# training.py:89, LunaTrainingApp.initTrainDl
def initTrainDl(self):
    train_ds = LunaDataset(
        val_stride=10,
        isValSet_bool=False,
    )

    batch_size = self.cli_args.batch_size
    if self.use_cuda:
        batch_size *= torch.cuda.device_count()

    train_dl = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=self.cli_args.num_workers,
        pin_memory=self.use_cuda, # 고정된 메모리 영역이 GPU로 빠르게 보내진다.
        )
    return train_dl

def main(self):
    log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

    train_dl = self.initTrainDl()
    val_dl = self.initValDl()

`DataLoader`는 각 샘플을 배칭할 수 있을 뿐 아니라, 데이터를 병렬로 불러올 수 있다. 또한 GPU 계산과 데이터 로딩을 동시에 진행시켜 주기 땜누에 프로젝트를 빠르게 실행할 수 있다.

## 첫 번째 경로 신경망 설계
---
8장에서 만들었던 신경망 설계를 기본으로 한다.

분류 모델에서는 tail, backbone, head로 구성된 구조가 흔하다. tail은 입력을 신경망에 넣기 전에 전처리 과정을 담당하는 부분이다. backbone에서 원하는 형태로 입력을 만들어야 하기 때문에 다른 부분과는 형태가 다른 경우가 많다. 본 프로젝트에서는 단순 배치 정규화 계층을 사용한다.

다음으로 backbone은 여러 계층을 가지는데 일반적으로는 연속된 블럭이 배치된다. 각 블럭은 동일한 셋의 계층을 가지며, 블럭을 거칠 때마다 필요한 입력 크기나 필터 수가 달라진다. 본 프로젝트에서는 두 개의 3x3 컨볼루션과 reLu 활성화 함수를 사용하고 마지막에는 맥스 풀링 연산을 사용한다. 아래 그림의 오른쪽 부분이 이에 해당한다.

블럭을 코드로 구현하면 아래와 같다.

![image](https://user-images.githubusercontent.com/76675506/190589244-8ef7b79e-c466-4f8c-af82-b8b27c2cd565.png)

In [ ]:
 # model.py:67, class LunaBlock

class LunaBlock(nn.Module):
    def __init__(self, in_channels, conv_channels):
        super().__init__()

        self.conv1 = nn.Conv3d(
            in_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(
            conv_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu2 = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool3d(2, 2)

    def forward(self, input_batch):
        block_out = self.conv1(input_batch)
        block_out = self.relu1(block_out)
        block_out = self.conv2(block_out)
        block_out = self.relu2(block_out)

        return self.maxpool(block_out)

신경망의 head 부분에서는 backbone의 출력을 받아 원하는 출력 형태로 바꾼다. 컨볼루션 신경망에서는 평탄화(flattening)하고 완전 연결 계층(fully connected layer)하는 역할을 하기도 한다. 이미지가 객체가 많은 형태거나 구별할 대상이 많다면 완전 연결 계층을 사용하는게 적합하다. 하지만 이 프로젝트에서는 두 가지로 분류하기 때문에 복잡하게 만들 필요 없이 하나의 평탄화 계층만 사용한다.

또한 시작할 때는 이런 식으로 단순하게 만들고 이유가 명확할 때 복잡성을 추가하는 것이 좋다.

아래 그림에서 우리가 사용하는 컨볼루션을 살펴볼 수 있다. 실제로 적용되는 블럭에서는 3x3x3 컨볼루션을 사용한다. 컨볼루션층이 쌓여있기 때문에 마지막 출력 복셀은 컨볼루션 커널의 크기보다 입력에 영향을 받는다.

![image](https://user-images.githubusercontent.com/76675506/190594445-d80b2364-9fb3-45ee-abf6-fae10fa2aee0.png)

전체 모델 구현은 아래 코드와 같다. 먼저 tail에서는 `nn.BatchNorm3d`를 이용해서 정규화(평균값 0, 표준편차 1)를 한다.

backbone에서는 네 개의 블럭을 반복한다. 각 블럭은 2x2x2 맥스 풀링으로 끝나기 때문에 네 개의 층을 거치면 이미지는 각 차원 별로 16배가 줄어든 형태가 된다. backbone을 거치고 나면 데이터는 2x3x3이 된다.

마지막으로 tail에서는 `nn.Softmax`로 마무리된다.

In [ ]:
class LunaModel(nn.Module):
    def __init__(self, in_channels=1, conv_channels=8):
        super().__init__()

        self.tail_batchnorm = nn.BatchNorm3d(1) # tail

        self.block1 = LunaBlock(in_channels, conv_channels)# backbone
        self.block2 = LunaBlock(conv_channels, conv_channels * 2)
        self.block3 = LunaBlock(conv_channels * 2, conv_channels * 4)
        self.block4 = LunaBlock(conv_channels * 4, conv_channels * 8)

        self.head_linear = nn.Linear(1152, 2) #head
        self.head_softmax = nn.Softmax(dim=1)

### 컨볼루션을 선형으로 변환하기
위처럼 모델을 정의하고 계속 진행하면 문제가 발생한다. `self.block4`의 출력을 완전 연결 계층에 넣을 수 없기 때문이다. 출력은 샘플마다 2x3x3 이미지에 64개 채널을 갖는데, 완전 연결 계층은 1차원 벡터를 받을 수 있기 때문이다. `forward` 메소드를 살펴보자.

아래 코드에서 완전 연결 계층에 전달하기 전에 `view`를 이용해 flatten을 해야한다. `forward` 메소드는 출력을 위해 logit과 softmax로 확률을 만든다. 훈련 중에는 `nn.CrossEntropyLoss` 계산을 위해 logit 값을 사용하고 실제로 분류할 때는 확률 값을 사용한다.

In [ ]:
#  model.py:50, LunaModel.forward

def forward(self, input_batch):
    bn_output = self.tail_batchnorm(input_batch)

    block_out = self.block1(bn_output)
    block_out = self.block2(block_out)
    block_out = self.block3(block_out)
    block_out = self.block4(block_out)

    conv_flat = block_out.view(
        block_out.size(0), # 배치 크기
        -1,
    )
    linear_output = self.head_linear(conv_flat)

    return linear_output, self.head_softmax(linear_output)

### 초기화
모델이 좋은 성능을 내려면 가중치, 편향값 등 여러 파라미터에 대해 주의할 점이 있다.

모든 가중치가 1보다 커지는 경우를 생각해보자. (residual connection이 없는 경우) 이 가중치 값으로 연산을 반복하면 출력 값이 매우 커지게 된다. 또, 모든 가중치가 1보다 작다면 출력 값을 사라지게 만든다. 역전파에서의 기울기도 동일한 문제가 발생한다.

이를 해결하는 방법은 신경망 가중치를 초기화하는 것이다. 이를 파이토치에서 제공하진 않으므로 우리가 직접 초기화해야 한다. `_init_weights`를 살펴보자. 코드를 자세하게 이해할 필요는 없고 이 메소드를 이용해 진행한다는 것만 기억하자.

Kaiming(he) initialization: https://brunch.co.kr/@kmbmjn95/37

In [ ]:
# model.py:30, LunaModel._init_weights

def _init_weights(self):
    for m in self.modules():
        if type(m) in {
            nn.Linear,
            nn.Conv3d,
            nn.Conv2d,
            nn.ConvTranspose2d,
            nn.ConvTranspose3d,
        }:# kaiming(he) initialization은 relu 연산을 위해 만들어진 초기화 방법이다.
            nn.init.kaiming_normal_(
                m.weight.data, a=0, mode='fan_out', nonlinearity='relu',
            )
            if m.bias is not None:
                fan_in, fan_out = \
                    nn.init._calculate_fan_in_and_fan_out(m.weight.data)
                bound = 1 / math.sqrt(fan_out)
                nn.init.normal_(m.bias, -bound, bound)

## 모델 훈련과 검증
---
지금까지 다룬 부분을 조립해서 동작해볼 차례다.


![image](https://user-images.githubusercontent.com/76675506/190607812-e53f4a3f-eb5f-4e75-8814-9aa83f6c2324.png)

In [ ]:
# training.py:137, LunaTrainingApp.main
def main(self):
    # ... 143 line
    for epoch_ndx in range(1, self.cli_args.epochs + 1):

        log.info("Epoch {} of {}, {}/{} batches of size {}*{}".format(
            epoch_ndx,
            self.cli_args.epochs,
            len(train_dl),
            len(val_dl),
            self.cli_args.batch_size,
            (torch.cuda.device_count() if self.use_cuda else 1),
        ))

        trnMetrics_t = self.doTraining(epoch_ndx, train_dl)
        self.logMetrics(epoch_ndx, 'trn', trnMetrics_t)

        valMetrics_t = self.doValidation(epoch_ndx, val_dl)
        self.logMetrics(epoch_ndx, 'val', valMetrics_t)

# ... 165 line
def doTraining(self, epoch_ndx, train_dl):
    self.model.train()
    trnMetrics_g = torch.zeros(
        METRICS_SIZE, # 빈 metric 배열 초기화
        len(train_dl.dataset),
        device=self.device,
    )

    batch_iter = enumerateWithEstimate( # 시간을 예측하며 배치 루프 설정
        train_dl,
        "E{} Training".format(epoch_ndx),
        start_ndx=train_dl.num_workers,
    )
    for batch_ndx, batch_tup in batch_iter:
        self.optimizer.zero_grad() # 남은 가중치 텐서 해제

        loss_var = self.computeBatchLoss(
            batch_ndx,
            batch_tup,
            train_dl.batch_size,
            trnMetrics_g
        )

        loss_var.backward()
        self.optimizer.step()

        # # This is for adding the model graph to TensorBoard.
        # if epoch_ndx == 1 and batch_ndx == 0:
        #     with torch.no_grad():
        #         model = LunaModel()
        #         self.trn_writer.add_graph(model, batch_tup[0], verbose=True)
        #         self.trn_writer.close()

    self.totalTrainingSamples_count += len(train_dl.dataset)

    return trnMetrics_g.to('cpu')

위 훈련 루프가 이전과 다른 점은 다음과 같다.
- `trnMetrics_g` 텐서가 훈련 중에 metric을 수집한다.(loggig)
- `train_dl` 데이터 로더를 직접 순회하지 않는다.
- 완료 시간 예측을 위한 `enumerateWithEstimate`를 사용한다.
- 실제 손실 계산은 `computeBatchLoss`에서 이뤄진다.

### computeBatchLoss 함수

`computeBatchLoss`는 훈련 루프와 검증 루프 모두에서 호츨된다. 샘플 배치에 대한 손실을 계산하고, 부가적으로 모델이 만들어내는 출력에 대한 정보도 계산해서 기록한다. 이를 이용해서 클래스별로 계산이 얼마나 정확한지 알 수 있다.


In [ ]:
def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g):
    input_t, label_t, _series_list, _center_list = batch_tup

    input_g = input_t.to(self.device, non_blocking=True)
    label_g = label_t.to(self.device, non_blocking=True)

    logits_g, probability_g = self.model(input_g)

    loss_func = nn.CrossEntropyLoss(reduction='none') # 'none'으로 했을 때 샘플별 손실값을 얻는다.
    loss_g = loss_func(
        logits_g,
        label_g[:,1], # 원핫 인코딩 클래스의 인덱스
    )

    # ... 238 line
    return loss_g.mean() # 샘플별 손실값을 단일값으로 합친다.

위 코드는 손실값을 배치 단위로 구하지 않는다. 대신 클래스별로 손실값이 들어있는 텐서를 샘플마다 얻는다. 이를 통해 개별 손실값을 추적할 수 있고 원하는 방식(예를 들면 클래스별로)으로 합칠 수 있다.

역전파 단계를 살펴보기 전에 추후 분석을 위해 샘플별 통계값을 기록하자. 이를 위해 파라미터로 받은 `metrics_g` 값을 이용한다.

In [ ]:
METRICS_LABEL_NDX=0
METRICS_PRED_NDX=1
METRICS_LOSS_NDX=2
METRICS_SIZE = 3

def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g):
        # ... 238 line
        start_ndx = batch_ndx * batch_size
        end_ndx = start_ndx + label_t.size(0)

        metrics_g[METRICS_LABEL_NDX, start_ndx:end_ndx] = \ # 우리가 사용하는 metric들은 gradient를 유지할 필요가 없으므로 deatch
            label_g[:,1].detach()
        metrics_g[METRICS_PRED_NDX, start_ndx:end_ndx] = \ # detach()는 텐서를 복사하는 방법으로 gradient가 전파되지 않는 텐서가 생성된다.
            probability_g[:,1].detach()
        metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = \
            loss_g.detach()

        return loss_g.mean()

### Validation loop

validation loop은 training loop과 비슷하다. 단지 read-only인 점만 다르다. 즉, loss 값을 리턴하지 않고 가중치 또한 업데이트 되지 않는다.

그 외에 다른 점은 동일하고 `with torch.no_grad()`로 인해 좀 더 빠르다.

In [ ]:
#  training.py:137, LunaTrainingApp.main
def main(self):
    for epoch_ndx in range(1, self.cli_args.epochs + 1):
        # ... 157 line
        valMetrics_t = self.doValidation(epoch_ndx, val_dl)
        self.logMetrics(epoch_ndx, 'val', valMetrics_t)

def doValidation(self, epoch_ndx, val_dl):
    with torch.no_grad():
        self.model.eval()   # 추론 모드
        valMetrics_g = torch.zeros(
            METRICS_SIZE,
            len(val_dl.dataset),
            device=self.device,
        )

        batch_iter = enumerateWithEstimate(
            val_dl,
            "E{} Validation ".format(epoch_ndx),
            start_ndx=val_dl.num_workers,
        )
        for batch_ndx, batch_tup in batch_iter:
            self.computeBatchLoss(
                batch_ndx, batch_tup, val_dl.batch_size, valMetrics_g)

    return valMetrics_g.to('cpu')

## metric 로깅하기

epoch 마다 성능을 metirc으로 기록하는 것은 중요하다. 이를 통해 어떤 점이 문제인지, 어떻게 고칠 수 있을지 생각해볼 수 있기 때문이다. 추후에는 로그 데이터를 바탕으로 epoch의 사이즈를 조작해본다.

In [ ]:
# training.py:251, LunaTrainingApp.logMetrics
def logMetrics(
        self,
        epoch_ndx, # 로깅하는 내역 나타내기
        mode_str, # train인지 valid인지 구분
        metrics_t, # trnMetrics_t 혹은 valMetrics_t
        classificationThreshold=0.5,
):

#### 마스크 구성하기

결절 샘플 혹은 비결절 샘플에 대해서만 metric을 제한하는 마스크를 만들어보자

In [ ]:
# training.py:264, LunaTrainingApp.logMetrics
        negLabel_mask = metrics_t[METRICS_LABEL_NDX] <= classificationThreshold # bool 값이 저장된 mask 생성
        negPred_mask = metrics_t[METRICS_PRED_NDX] <= classificationThreshold

        posLabel_mask = ~negLabel_mask
        posPred_mask = ~negPred_mask


        neg_count = int(negLabel_mask.sum())
        pos_count = int(posLabel_mask.sum())

        neg_correct = int((negLabel_mask & negPred_mask).sum())
        pos_correct = int((posLabel_mask & posPred_mask).sum())

        metrics_dict = {}
        metrics_dict['loss/all'] = \
            metrics_t[METRICS_LOSS_NDX].mean()
        metrics_dict['loss/neg'] = \
            metrics_t[METRICS_LOSS_NDX, negLabel_mask].mean() # 손실값을 클래스 별로도 저장해둔다. 이렇게 하면 특정 클래스에 대해 예측률이 낮을 때 개선하기 쉽다.
        metrics_dict['loss/pos'] = \
            metrics_t[METRICS_LOSS_NDX, posLabel_mask].mean()

        metrics_dict['correct/all'] = (pos_correct + neg_correct) \
            / np.float32(metrics_t.shape[1]) * 100
        metrics_dict['correct/neg'] = neg_correct / np.float32(neg_count) * 100
        metrics_dict['correct/pos'] = pos_correct / np.float32(pos_count) * 100


위에서 계산한 결과를 `log.info`로 저장한다.

In [ ]:
  log.info(
            ("E{} {:8} {loss/all:.4f} loss, "
                 + "{correct/all:-5.1f}% correct, "
            ).format(
                epoch_ndx,
                mode_str,
                **metrics_dict,
            )
        )
        log.info(
            ("E{} {:8} {loss/neg:.4f} loss, "
                 + "{correct/neg:-5.1f}% correct ({neg_correct:} of {neg_count:})"
            ).format(
                epoch_ndx,
                mode_str + '_neg',
                neg_correct=neg_correct,
                neg_count=neg_count,
                **metrics_dict,
            )
        )
        log.info(
            ("E{} {:8} {loss/pos:.4f} loss, "
                 + "{correct/pos:-5.1f}% correct ({pos_correct:} of {pos_count:})"
            ).format(
                epoch_ndx,
                mode_str + '_pos',
                pos_correct=pos_correct,
                pos_count=pos_count,
                **metrics_dict,
            )
        )

## 훈련 스크립트 실행